In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

**Multiclass Neural Network**







In [2]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving Iris.csv to Iris (3).csv


In [3]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name = fn , length = len(uploaded[fn])))

User uploaded file "Iris.csv" with length 5107 bytes


In [4]:
import io
## https://github.com/yangzhangalmo/pytorch-iris/blob/master/main.py
df = pd.read_csv(io.StringIO(uploaded['Iris.csv'].decode('utf-8')))
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
# transform species to numerics for Target Variables..
df.loc[df.Species=='Iris-setosa', 'Species'] = 0
df.loc[df.Species=='Iris-versicolor', 'Species'] = 1
df.loc[df.Species=='Iris-virginica', 'Species'] = 2

In [6]:
## Train-Test Split
from sklearn.model_selection import train_test_split
features = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']
X_train, X_test, y_train, y_test = train_test_split(df[features], df['Species'], test_size=0.2, random_state=42,stratify = df['Species'])

X_train = torch.tensor(X_train.values,dtype=torch.float32)
X_test = torch.tensor(X_test.values,dtype=torch.float32)
y_train = torch.tensor(y_train.values,dtype=torch.long)
y_test = torch.tensor(y_test.values,dtype=torch.long)
print('Shape of--',X_train.shape,X_test.shape,y_train.shape,y_test.shape)

Shape of-- torch.Size([120, 4]) torch.Size([30, 4]) torch.Size([120]) torch.Size([30])


In [7]:
print('Analogy of Dataset\n')
print('x_data',X_train[0:5])
print('y_data',y_train[0:5])

Analogy of Dataset

x_data tensor([[4.4000, 2.9000, 1.4000, 0.2000],
        [4.9000, 2.5000, 4.5000, 1.7000],
        [6.8000, 2.8000, 4.8000, 1.4000],
        [4.9000, 3.1000, 1.5000, 0.1000],
        [5.5000, 2.5000, 4.0000, 1.3000]])
y_data tensor([0, 2, 1, 0, 1])


In [0]:
class FNN(nn.Module):
    def __init__(self,InputShape,HiddenUnits_1,HiddenUnits_2,OutputShape):
        super(FNN,self).__init__()
        self.fc1 = nn.Linear(InputShape,HiddenUnits_1)
        self.fc2 = nn.Linear(HiddenUnits_1,HiddenUnits_2)
        self.fc3 = nn.Linear(HiddenUnits_2,OutputShape)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self,x):
      x = self.fc1(x)
      x = torch.relu(x)
      x = self.fc2(x)
      x = torch.relu(x)
      x = self.fc3(x)
      x = self.softmax(x)

      return x

In [9]:
model = FNN(4,32,16,3) ## 4 is input ## 32 and 16 are hidden units and 3 are number of classes
print("Archeitecture of Model\n",model)

Archeitecture of Model
 FNN(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)


In [10]:
criteria = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

n_iter = 100

for epoch in range(n_iter):
  optimizer.zero_grad()
  out = model(X_train)
  loss = criteria(out, y_train)
  loss.backward()
  optimizer.step()

  if epoch % 10 == 0:
    print('epoch',epoch,'loss',loss.item(),loss.data)

epoch 0 loss 1.0914219617843628 tensor(1.0914)
epoch 10 loss 1.0844645500183105 tensor(1.0845)
epoch 20 loss 1.078510046005249 tensor(1.0785)
epoch 30 loss 1.073340892791748 tensor(1.0733)
epoch 40 loss 1.068795919418335 tensor(1.0688)
epoch 50 loss 1.0647331476211548 tensor(1.0647)
epoch 60 loss 1.0610103607177734 tensor(1.0610)
epoch 70 loss 1.0575206279754639 tensor(1.0575)
epoch 80 loss 1.0541799068450928 tensor(1.0542)
epoch 90 loss 1.0509170293807983 tensor(1.0509)


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

predict_out = model(X_test)
_, predict_y = torch.max(predict_out, 1)

print ('prediction accuracy', accuracy_score(y_test.data, predict_y.data))

prediction accuracy 0.43333333333333335


In [12]:
predict_y.data

tensor([0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

**Regression based Neural Network**

In [13]:

from sklearn.datasets import load_boston
boston_dataset = load_boston()

#dict_keys(['data', 'target', 'feature_names', 'DESCR'])

df_boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df_boston['MEDV'] = boston_dataset.target ## adding into dataset.
df_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [14]:
## target
#### https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379

import numpy as np
X = pd.DataFrame(np.c_[df_boston['LSTAT'], df_boston['RM'],df_boston['AGE'],df_boston['INDUS']], columns = ['LSTAT','RM','AGE','INDUS'])
Y = df_boston['MEDV']

## train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

X_train = torch.tensor(X_train.values,dtype=torch.float32)
X_test = torch.tensor(X_test.values,dtype=torch.float32)
### Changing Dimension of Target
##y_train = y_train.view(len(y_train),1) ###
y_train = torch.tensor(y_train.values,dtype=torch.float32)
##y_test = y_test.view(len(y_train),1) ###
y_test = torch.tensor(y_test.values,dtype=torch.float32)
print('Shape of--',X_train.shape,X_test.shape,y_train.shape,y_test.shape)

Shape of-- torch.Size([339, 4]) torch.Size([167, 4]) torch.Size([339]) torch.Size([167])


In [15]:
print('X_train',X_train[0:5])
print('y_train',y_train[0:5])

X_train tensor([[18.0300,  6.1850, 96.7000, 18.1000],
        [14.8100,  5.8130, 90.3000,  8.1400],
        [19.1500,  6.1720, 96.1000,  7.8700],
        [13.3500,  5.9830, 83.5000, 27.7400],
        [12.2700,  6.4740, 97.1000,  8.5600]])
y_train tensor([14.6000, 16.6000, 27.1000, 20.1000, 19.8000])


In [0]:
class Linear_FNN(nn.Module):
  def __init__(self,inputsize,hidden_1,hidden_2,outputsize):
    super(Linear_FNN,self).__init__()
    self.fc1 = nn.Linear(inputsize,hidden_1)
    self.fc2 = nn.Linear(hidden_1,hidden_2)
    self.predict = nn.Linear(hidden_2,outputsize)

  def forward(self,x):
    x = self.fc1(x)
    x = torch.relu(x)
    x = self.fc2(x)
    x = torch.sigmoid(x)
    out = self.predict(x)

    return out


In [0]:
hidden_1 = 8
hidden_2 = 16
model_linear = Linear_FNN(X_train.shape[1],hidden_1,hidden_2,1)

In [18]:
criteria = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

n_iter = 100
for epoch in range(n_iter):
  optimizer.zero_grad()
  out = model(X_train)
  loss = criteria(out, y_train.view(len(y_train),1))
  loss.backward()
  optimizer.step()

  if epoch % 10 == 0:
    print('epoch',epoch,'loss',loss.item(),loss.data)

epoch 0 loss 600.8104858398438 tensor(600.8105)
epoch 10 loss 600.8088989257812 tensor(600.8089)
epoch 20 loss 600.80615234375 tensor(600.8062)
epoch 30 loss 600.7999877929688 tensor(600.8000)
epoch 40 loss 600.69140625 tensor(600.6914)
epoch 50 loss 600.6824951171875 tensor(600.6825)
epoch 60 loss 600.6736450195312 tensor(600.6736)
epoch 70 loss 600.6650390625 tensor(600.6650)
epoch 80 loss 600.6968994140625 tensor(600.6969)
epoch 90 loss 600.6864624023438 tensor(600.6865)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([339, 1])) that is different to the input size (torch.Size([339, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


**Binary Loss Neural Network**

In [0]:
## https://www.datahubbs.com/deep-learning-101-first-neural-network-with-pytorch/
## https://gist.github.com/santi-pdp/d0e9002afe74db04aa5bbff6d076e8fe

X = torch.tensor([[1], [2], [3], [4],[5],[6],[7],[8]], dtype=torch.float32)
Y = torch.tensor([[1.], [0.], [1.], [0.],[1.],[0],[1],[0]], dtype=torch.float32)

In [0]:
class log_FNN(nn.Module):
  def __init__(self,i_size,hidden,o_size):
    super(log_FNN,self).__init__()

    self.fc1 = nn.Linear(i_size,hidden)
    self.fc2 = nn.Linear(hidden,o_size)
  
  def forward(self,x):
    x = self.fc1(x)
    x = torch.relu(x)
    x = self.fc2(x)
    out = torch.sigmoid(x)

    return out


In [0]:
model = log_FNN(1,16,1)

In [22]:
loss_torch = nn.BCELoss(reduction='mean') ## Use Binary cross Entropyn loss
optimizer = torch.optim.SGD(model.parameters(),lr = .001)
n_iter = 10


for epoch in range(n_iter):
  optimizer.zero_grad()
  out = model(X)
  loss = criteria(out, Y)
  loss.backward()
  optimizer.step()

  if epoch % 2 == 0:
    print('epoch',epoch,'loss',loss.item(),loss.data)


epoch 0 loss 0.3728271424770355 tensor(0.3728)
epoch 2 loss 0.37208664417266846 tensor(0.3721)
epoch 4 loss 0.37133875489234924 tensor(0.3713)
epoch 6 loss 0.37058332562446594 tensor(0.3706)
epoch 8 loss 0.36982035636901855 tensor(0.3698)
